# Remote Service Activity Detected via SVCCTL named pipe
Detects remote remote service activity via remote access to the svcctl named pipe

## Rule Content
```
- title: Remote Service Activity Detected via SVCCTL named pipe
  id: 586a8d6b-6bfe-4ad9-9d78-888cd2fe50c3
  description: Detects remote remote service activity via remote access to the svcctl
    named pipe
  author: Samir Bousseaden
  references:
  - https://blog.menasec.net/2019/03/threat-hunting-26-remote-windows.html
  tags:
  - attack.lateral_movement
  - attack.persistence
  logsource:
    product: windows
    service: security
    description: The advanced audit policy setting "Object Access > Audit Detailed
      File Share" must be configured for Success/Failure
    category: null
  detection:
    selection:
      EventID: 5145
      ShareName: \\*\IPC$
      RelativeTargetName: svcctl
      Accesses: '*WriteData*'
    condition: selection
  falsepositives:
  - pentesting
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"5145" AND share_name.keyword:\\*\\IPC$ AND share_relative_target_name:"svcctl" AND Accesses.keyword:*WriteData*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()